In [ ]:
#only for google colab
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/Colab_Notebooks/praknlp'
drive.mount('/content/gdrive', force_remount=True)
os.chdir(gdrive_path)

Mounted at /content/gdrive


In [ ]:
!pip install "tensorflow==1.15.0"
!pip install --upgrade tensorflow_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 503 kB 56.8 MB/s 
     |████████████████████████████████| 3.8 MB 36.7 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=c1ce05aabe2de64413b875f3c046c0d09e035f249d1b5acbea9f4049042c747a
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting un

In [ ]:
import pandas as pd
import numpy as np
import spacy
import nltk
import time
import pickle
import torch
import json
import re

import tensorflow_hub as hub
import tensorflow as tf

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#import recipe instructions
final_datalist = []

# open file and read the content in a list
with open(r'./data/1100kinstruction.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        final_datalist.append(x)

In [ ]:
#import list of all ingredients used in recipe1m
with open('./ELMo_embeddings/used_ingredients.json','r') as fp:
  used_ing = json.load(fp)
print(len(used_ing))

4418


In [ ]:
used_ing_spaced = []
for i in used_ing:
  res = re.sub('_',' ',i)
  used_ing_spaced.append(i)
print(len(used_ing_spaced))

4418


In [ ]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
#import elmo modelling here

elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [ ]:
#save desired number of recipes in data
data = final_datalist[:1000]

In [ ]:
ingredient_embed_dict = {}
ingredient_freq = {}
rno = 0

In [ ]:
"""
with open('/content/gdrive/MyDrive/Colab_Notebooks/praknlp/ingredient_embed.pkl', 'rb') as fil:
  ingredient_embed_dict = pickle.load(fil)
with open('/content/gdrive/MyDrive/Colab_Notebooks/praknlp/ingredient_freq.pkl', 'rb') as f:
  ingredient_freq = pickle.load(f)
"""

In [ ]:
#all code in looping iteration for a set of recipes
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

stopwords = stopwords.words('english')

for recipe in data:
  rno+=1
  print('Recipe number in progress: ', rno)
  #remove punctuation
  recipe = re.sub(r'\.\ \)\.','\)\.',recipe)
  recipe = re.sub(r'[^\.\-\!\?\w\s]','',recipe)
  recipe = re.sub(r'[^\.\-\w\s]','.',recipe)
  recipe = re.sub(r'[^\.\w\s]',' ',recipe)
  recipe = re.sub(r'\d+', '', recipe)
  recipe = recipe.lower()
  #print(recipe)

  #tokenize sentences
  sent = sent_tokenize(recipe)
  nos = len(sent) #returns number of sentences
  #print('No of sentences in recipe is: ',nos)

  #remove stopwords and reformulate sentence
  sent_tokens = []
  for i in sent:
    c = []
    z = word_tokenize(i)
    c = [j for j in z if j.lower() not in stopwords]
    d = TreebankWordDetokenizer().detokenize(c)
    sent_tokens.append(d)
  sent_nostop = TreebankWordDetokenizer().detokenize(sent_tokens) #replace e with sent_nostop, detokenized sentence here for ease of use ahead
  #print(sent_nostop)

  #tokenize words
  words = word_tokenize(sent_nostop)

  #check which items are present in the ingredients list
  items = []
  for j in words:
    if j in used_ing and j not in items:
      items.append(j)
  #print(items)
  
  #apply elmo on this cleaned sentence
  embed = elmo(sent_tokens, signature="default", as_dict=True)["elmo"]
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    embeddings = sess.run(embed) #works for cpu, fails for gpu
  
  #obtain ingredient embedding here
  fs=0
  fw=0
  #while fs<=cs: # this is not needed in current case, can be used to optimize for speed
  for word in words:
    if word in items: #embedding of ingredient processed here
      q = embeddings[fs,fw]
      if word in ingredient_embed_dict: #update embedding
        x = ingredient_embed_dict.get(word)
        x+=q
        ingredient_embed_dict[word] = x
        ingredient_freq[word]+=1
        #this will be >1, try finding the occurances of how many times this trigger occured
      else: #add new embedding
        ingredient_embed_dict[word] = q
        ingredient_freq[word] = 1
    elif word == '.':
      #jump to next sentence here
      fs+=1
      fw=0
  #end of recipe iteration
  if rno%100 == 0: #save to pickle
    print('Pickling after ' + str(rno) + 'th recipe')
    with open('./ELMo_model/ingredient_embed.pkl', 'wb') as fil:
      pickle.dump(ingredient_embed_dict, fil)
    with open('./ELMo_model/ingredient_freq.pkl', 'wb') as f:
      pickle.dump(ingredient_freq, f)

Recipe number in progress:  1
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Recipe number in progress:  2
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Recipe number in progress:  3
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

Recipe number in progress:  199
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Recipe number in progress:  200
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Pickling after 200th recipe


In [ ]:
print(ingredient_embed_dict)

{'butter': array([ 52.972763,  18.181664, 123.35716 , ...,  23.961966, -59.595367,
        93.25452 ], dtype=float32), 'oil': array([ 52.972763,  18.181664, 123.35716 , ...,  23.961966, -59.595367,
        93.25452 ], dtype=float32), 'penne': array([ 0.53989244, -0.3374067 ,  0.61758333, ...,  0.2126518 ,
       -0.538821  ,  0.0256914 ], dtype=float32), 'pasta': array([ 48.777866,  12.279967,  57.47175 , ...,  10.192367, -41.21441 ,
        50.12333 ], dtype=float32), 'water': array([ 48.777866,  12.279967,  57.47175 , ...,  10.192367, -41.21441 ,
        50.12333 ], dtype=float32), 'sauce': array([ 21.424496 ,   8.666265 ,  46.52833  , ...,   5.5908265,
       -16.242468 ,  35.324684 ], dtype=float32), 'sprinkle': array([ 15.6909  ,  36.71889 ,  50.81507 , ...,  11.945403, -24.75973 ,
        25.543186], dtype=float32), 'chili': array([ 15.6909  ,  36.71889 ,  50.81507 , ...,  11.945403, -24.75973 ,
        25.543186], dtype=float32), 'cheese': array([ 27.61032 ,  20.60123 ,  51.6907

In [ ]:
print(ingredient_freq)

{'butter': 104, 'oil': 97, 'penne': 1, 'pasta': 36, 'water': 87, 'sauce': 76, 'sprinkle': 56, 'chili': 7, 'cheese': 94, 'flour': 59, 'milk': 41, 'salt': 113, 'garlic': 46, 'chipotle': 1, 'liquid': 12, 'macaroni': 5, 'celery': 12, 'green': 18, 'pepper': 89, 'pimento': 1, 'mayonnaise': 14, 'salad': 17, 'dressing': 15, 'vinegar': 14, 'dill': 3, 'lettuce': 3, 'tomato': 10, 'olive': 18, 'garnish': 16, 'basil': 10, 'corn': 18, 'potato': 4, 'jello': 2, 'watermelon': 2, 'crust': 20, 'toast': 6, 'coconut': 10, 'crisp': 8, 'beef': 14, 'lemon': 37, 'juice': 41, 'soy': 12, 'cornstarch': 8, 'pineapple': 5, 'orange': 11, 'rice': 16, 'cashew': 1, 'chicken': 79, 'tea': 2, 'meat': 28, 'sugar': 95, 'vanilla': 22, 'zucchini': 14, 'loaf': 11, 'red': 14, 'salmon': 10, 'fillet': 2, 'ginger': 10, 'black': 12, 'fish': 8, 'la': 1, 'cookie': 15, 'parmesan': 11, 'topping': 8, 'banana': 2, 'kiwi': 1, 'cinnamon': 19, 'yogurt': 4, 'fruit': 9, 'fennel': 9, 'pork': 10, 'wine': 14, 'broth': 20, 'roast': 12, 'hot': 23,